In [7]:
from IPython.display import display, Image, clear_output, HTML
import time
import random
random.seed(1) #makes the test reproducible
import keyboard
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests

In [8]:
def send_to_google_form(data_dict, form_url):
    '''
    Sends the dataframe of gathered information and results to a google form.
    
    Parameters
    ----------
    data_dict: dict
        A dictionary containing the data to be sent to the Google Form.
        The keys are: user_id, gender, filename, correct, time, total_time, total_correct, total_incorrect.
    form_url: str
        The URL to the google form to which the data will be sent.
    
    Returns
    -------
    bool
        True if data is sent to form successfully. False if not.
    '''

    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [9]:
def test_instructions():
    
    '''
    Prints the test instructions in a HTML format, and collects user information
    
    Parameters
    ----------
    none 
    
    Returns
    -------
    str
        Returns the user ID.
    str
        Returns the user's gender ('male', 'female' or 'other').
    '''

    
    data_consent_info = """DATA CONSENT INFORMATION:

Please read:

We wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ") 

    if result == "yes": 
        print("Thanks for your participation.")
        print("Please contact a.fedorec@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")
    
    else: 
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))
        
    id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifier would be CBTC
"""

    print(id_instructions)
    user_id = input("> ")

    print("User entered id:", user_id)
    
    enter_gender = '''
    
Enter your gender.

Enter male, female or other.'''
    
    print(enter_gender)
    gender = input('> ')
    
    if gender.lower() in ("male", "female", "other"):
        print("User entered gender:", gender)
    else: 
        # end code execution by raising an exception
        raise(Exception("User did not enter a gender."))
    
    
    
    

    style = "color : green; font-size : 60px;"
    
    
    first_instruction = HTML(f'<h1 style = {style}>Welcome to the Approximate Number Sense test.</h1>')
    second_instruction = HTML(f'<h1 style = {style}>A series of images with dots will be displayed (followed by an empty image if you need more time to answer).</h1>')
    third_instruction = HTML(f'<h1 style = {style}>You must decide which side has more dots (left or right) within 3 seconds.</h1>')
    fourth_instruction = HTML(f'<h1 style = {style}>To submit your answer use the left and right keys ON THE KEYBOARD (not the mouse).</h1>')
    fifth_instruction = HTML(f'<h1 style = {style}>FROM THE MOMENT THE IMAGE APPEARS ON THE SCREEN you may enter your answer.</h1>')
    final_instruction = HTML(f'<h1 style = {style}>Good luck! The test will begin now...</h1>')
    instructions_list = [first_instruction, second_instruction, third_instruction, fourth_instruction, fifth_instruction, final_instruction]                         
     
    for instruction in instructions_list:
        display(instruction)
        time.sleep(3)
        clear_output(wait = False)

    return user_id, gender

In [10]:
def correct_or_incorrect(image, images_dict):
    '''
    Determines if the keyboard has been pressed, and if it has then it determines if the user has selected the correct answer.
    
    Parameters
    ----------
    image: str
        The name of the image that is being displayed.
    images_dict: dict
        A dictionary of images, where the keys are the name of the image and the values are 'left' or 'right' depending on which side of the image has more dots.
    
    Returns
    -------
    bool
        Returns 'correct' which is equal to either True, False or None.
    '''
    
    if keyboard.is_pressed('left'):
        if images_dict[image] == 'left':
            correct = True
        elif images_dict[image] == 'right':
            correct = False
                    
    elif keyboard.is_pressed('right'):
        if images_dict[image] == 'right':
            correct = True
        elif images_dict[image] == 'left':
            correct = False
                     
    else:
        correct = None
          
    return correct
    

In [11]:
def ans_test(images_list, images_dict, empty_image):
    '''
    Displays each image and records (for each image) the file name, the time taken and whether the question has been responded to correctly.
    Records the overall time taken, number of correct and incorrect answers.
    
    Parameters
    ----------
    images_list: list
        The list of Image objects to be displayed.
    images_dict: dict
        A dictionary of images, where the keys are the name of the image and the values are 'left' or 'right' depending on which side of the image has more dots.
    empty_image: Image
        An Image object showing an empty grid.
        It displayed when the user does not respond within 0.75 seconds, and up until 3 seconds have passed since the non-empty image has been shown.
    
    Returns
    -------
    dict
        Returns 'results_dict', where the keys are: filename, correct, time, total_time, total_correct, total_incorrect.
        The values are strings.
    '''
    
    total_correct = 0
    total_incorrect = 0
    total_time = 0
    
    question_number = 1

    results_dict = {'filename': [], 'correct': [], 'time': [], 'total_time': [], 'total_correct': [], 'total_incorrect': []}
    
    for image in images_list:
        
        image_name = image.filename #allows you to use the Image object's name
        results_dict['filename'].append(image_name) #adds the name of the Image object to the filename list
        
        print(question_number, '/ 64')
        question_number += 1 #updates the question number so that when the next image is shown the question number is correct
        
        time_taken = 0
        start_time = time.time()
        
        correct = None
        display(image)
        
        while time.time() - start_time <= 0.75 and correct == None: 
            #if no answer has been submitted and the time is within 0.75 seconds, the image is continued to be displayed and an answer continues to be checked for
            correct = correct_or_incorrect(image, images_dict)
            
            
        clear_output(wait = False)
        
        if correct != None:
            time_taken = time.time() - start_time
            results_dict['time'].append(time_taken)
            
        else:
            display(empty_image) #if no answer has been detected past 0.75 seconds, display the empty template
            
            while (time.time() - start_time <= 2.75) and correct == None:
                #if the time is past 0.75 seconds and before 3 seconds and no answer has been detected, the empty template is shown and an answer continues to be checked for
                correct = correct_or_incorrect(image, images_dict)
                
                
            clear_output(wait = False)
            
            if time_taken == 0: 
                #if the time taken has not been recorded yet, the time taken is measured and appended to time_taken list
                time_taken = time.time() - start_time
                results_dict['time'].append(time_taken)
                
        total_time += time_taken
            
        if correct == True:
            total_correct += 1
            results_dict['correct'].append('correct')
        else:
            #if the question was answered wrongly or not at all, 'incorrect' is appended to the correct list and 1 is added to total_incorrect
            total_incorrect += 1
            results_dict['correct'].append('incorrect')
            
        
        time.sleep(1.5) #wait 1.5 seconds after the result is recorded or the question completes without a response before showing the next image
    
    results_dict['total_time'] = total_time
    results_dict['total_correct'] = total_correct
    results_dict['total_incorrect'] = total_incorrect
    print(f'Your score was {total_correct} out of 64.')
    
    return results_dict

In [ ]:
def main():
    '''
    Executes the main function of the Approximate Number Sense test.

    This function conducts the test, collects user information, displays images, records responses,
    calculates scores, and sends the results to a Google Form.
    
    Parameters
    ----------
    none
    
    Returns
    -------
    none
    
    '''
    
    #Image objects are named here
    empty_image = Image(filename = 'empty_image.png', width = 1000) 
    
    image_9_12_r1 = Image(filename = '9_12_r1.png', width = 1000)
    image_9_12_r2 = Image(filename = '9_12_r2.png', width = 1000)
    image_9_12_r3 = Image(filename = '9_12_r3.png', width = 1000)
    image_9_12_r4 = Image(filename = '9_12_r4.png', width = 1000)
    
    image_12_9_l1 = Image(filename = '12_9_l1.png', width = 1000)
    image_12_9_l2 = Image(filename = '12_9_l2.png', width = 1000)
    image_12_9_l3 = Image(filename = '12_9_l3.png', width = 1000)
    image_12_9_l4 = Image(filename = '12_9_l4.png', width = 1000)
    
    image_14_12_l1 = Image(filename = '14_12_l1.png', width = 1000)
    image_14_12_l2 = Image(filename = '14_12_l2.png', width = 1000)
    image_14_12_l3 = Image(filename = '14_12_l3.png', width = 1000)
    image_14_12_l4 = Image(filename = '14_12_l4.png', width = 1000)
    
    image_12_14_r1 = Image(filename = '12_14_r1.png', width = 1000)
    image_12_14_r2 = Image(filename = '12_14_r2.png', width = 1000)
    image_12_14_r3 = Image(filename = '12_14_r3.png', width = 1000)
    image_12_14_r4 = Image(filename = '12_14_r4.png', width = 1000)
    
    image_15_20_r1 = Image(filename = '15_20_r1.png', width = 1000)
    image_15_20_r2 = Image(filename = '15_20_r2.png', width = 1000)
    image_15_20_r3 = Image(filename = '15_20_r3.png', width = 1000)
    image_15_20_r4 = Image(filename = '15_20_r4.png', width = 1000)
    
    image_20_15_l1 = Image(filename = '20_15_l1.png', width = 1000)
    image_20_15_l2 = Image(filename = '20_15_l2.png', width = 1000)
    image_20_15_l3 = Image(filename = '20_15_l3.png', width = 1000)
    image_20_15_l4 = Image(filename = '20_15_l4.png', width = 1000)
    
    image_18_16_l1 = Image(filename = '18_16_l1.png', width = 1000)
    image_18_16_l2 = Image(filename = '18_16_l2.png', width = 1000)
    image_18_16_l3 = Image(filename = '18_16_l3.png', width = 1000)
    image_18_16_l4 = Image(filename = '18_16_l4.png', width = 1000)
    
    image_16_18_r1 = Image(filename = '16_18_r1.png', width = 1000)
    image_16_18_r2 = Image(filename = '16_18_r2.png', width = 1000)
    image_16_18_r3 = Image(filename = '16_18_r3.png', width = 1000)
    image_16_18_r4 = Image(filename = '16_18_r4.png', width = 1000)
    
    image_20_18_l1 = Image(filename = '20_18_l1.png', width = 1000)
    image_20_18_l2 = Image(filename = '20_18_l2.png', width = 1000)
    image_20_18_l3 = Image(filename = '20_18_l3.png', width = 1000)
    image_20_18_l4 = Image(filename = '20_18_l4.png', width = 1000)
    
    image_18_20_r1 = Image(filename = '18_20_r1.png', width = 1000)
    image_18_20_r2 = Image(filename = '18_20_r2.png', width = 1000)
    image_18_20_r3 = Image(filename = '18_20_r3.png', width = 1000)
    image_18_20_r4 = Image(filename = '18_20_r4.png', width = 1000)
    
    image_21_18_l1 = Image(filename = '21_18_l1.png', width = 1000)
    image_21_18_l2 = Image(filename = '21_18_l2.png', width = 1000)
    image_21_18_l3 = Image(filename = '21_18_l3.png', width = 1000)
    image_21_18_l4 = Image(filename = '21_18_l4.png', width = 1000)
    
    image_18_21_r1 = Image(filename = '18_21_r1.png', width = 1000)
    image_18_21_r2 = Image(filename = '18_21_r2.png', width = 1000)
    image_18_21_r3 = Image(filename = '18_21_r3.png', width = 1000)
    image_18_21_r4 = Image(filename = '18_21_r4.png', width = 1000)
    
    image_9_10_r1 = Image(filename = '9_10_r1.png', width = 1000)
    image_9_10_r2 = Image(filename = '9_10_r2.png', width = 1000)
    image_9_10_r3 = Image(filename = '9_10_r3.png', width = 1000)
    image_9_10_r4 = Image(filename = '9_10_r4.png', width = 1000)
    
    image_10_9_l1 = Image(filename = '10_9_l1.png', width = 1000)
    image_10_9_l2 = Image(filename = '10_9_l2.png', width = 1000)
    image_10_9_l3 = Image(filename = '10_9_l3.png', width = 1000)
    image_10_9_l4 = Image(filename = '10_9_l4.png', width = 1000)
    
    image_16_12_l1 = Image(filename = '16_12_l1.png', width = 1000)
    image_16_12_l2 = Image(filename = '16_12_l2.png', width = 1000)
    image_16_12_l3 = Image(filename = '16_12_l3.png', width = 1000)
    image_16_12_l4 = Image(filename = '16_12_l4.png', width = 1000)
    
    image_12_16_r1 = Image(filename = '12_16_r1.png', width = 1000)
    image_12_16_r2 = Image(filename = '12_16_r2.png', width = 1000)
    image_12_16_r3 = Image(filename = '12_16_r3.png', width = 1000)
    image_12_16_r4 = Image(filename = '12_16_r4.png', width = 1000)
    
    all_images = [image_9_12_r1, image_9_12_r2, image_9_12_r3, image_9_12_r4, image_12_9_l1, image_12_9_l2, image_12_9_l3, image_12_9_l4,
                  image_14_12_l1, image_14_12_l2, image_14_12_l3, image_14_12_l4, image_12_14_r1, image_12_14_r2, image_12_14_r3, image_12_14_r4,
                  image_15_20_r1, image_15_20_r2, image_15_20_r3, image_15_20_r4, image_20_15_l1, image_20_15_l2, image_20_15_l3, image_20_15_l4,
                  image_18_16_l1, image_18_16_l2, image_18_16_l3, image_18_16_l4, image_16_18_r1, image_16_18_r2, image_16_18_r3, image_16_18_r4,
                  image_20_18_l1, image_20_18_l2, image_20_18_l3, image_20_18_l4, image_18_20_r1, image_18_20_r2, image_18_20_r3, image_18_20_r4,
                  image_21_18_l1, image_21_18_l2, image_21_18_l3, image_21_18_l4, image_18_21_r1, image_18_21_r2, image_18_21_r3, image_18_21_r4,
                  image_16_12_l1, image_16_12_l2, image_16_12_l3, image_16_12_l4, image_12_16_r1, image_12_16_r2, image_12_16_r3, image_12_16_r4,
                  image_9_10_r1, image_9_10_r2, image_9_10_r3, image_9_10_r4, image_10_9_l1, image_10_9_l2, image_10_9_l3, image_10_9_l4]
    
    images_list = all_images.copy()
    random.shuffle(images_list)

    images_dict = {image_9_12_r1 : 'right', image_9_12_r2 : 'right', image_9_12_r3 : 'right', image_9_12_r4 : 'right', 
                   image_12_9_l1 : 'left', image_12_9_l2 : 'left', image_12_9_l3 : 'left', image_12_9_l4 : 'left',
                   image_14_12_l1 : 'left', image_14_12_l2 : 'left', image_14_12_l3 : 'left',  image_14_12_l4 : 'left',
                   image_12_14_r1 : 'right', image_12_14_r2 : 'right', image_12_14_r3 : 'right', image_12_14_r4 : 'right',
                   image_15_20_r1 : 'right', image_15_20_r2 : 'right', image_15_20_r3 : 'right', image_15_20_r4 : 'right',
                   image_20_15_l1 : 'left', image_20_15_l2 : 'left', image_20_15_l3 : 'left', image_20_15_l4 : 'left',
                   image_18_16_l1 : 'left', image_18_16_l2 : 'left', image_18_16_l3 : 'left',  image_18_16_l4 : 'left',
                   image_16_18_r1 : 'right', image_16_18_r2 : 'right', image_16_18_r3 : 'right', image_16_18_r4 : 'right',
                   image_20_18_l1 : 'left', image_20_18_l2 : 'left', image_20_18_l3 : 'left', image_20_18_l4 : 'left',
                   image_18_20_r1 : 'right', image_18_20_r2 : 'right', image_18_20_r3 : 'right', image_18_20_r4 : 'right',
                   image_18_21_r1 : 'right', image_18_21_r2 : 'right', image_18_21_r3 : 'right', image_18_21_r4 : 'right', 
                   image_21_18_l1 : 'left', image_21_18_l2 : 'left', image_21_18_l3 : 'left',  image_21_18_l4 : 'left',
                   image_9_10_r1 : 'right', image_9_10_r2 : 'right', image_9_10_r3 : 'right', image_9_10_r4 : 'right', 
                   image_10_9_l1 : 'left', image_10_9_l2 : 'left', image_10_9_l3 : 'left', image_10_9_l4 : 'left',
                   image_16_12_l1 : 'left', image_16_12_l2 : 'left', image_16_12_l3 : 'left', image_16_12_l4 : 'left',
                   image_12_16_r1 : 'right', image_12_16_r2 : 'right', image_12_16_r3 : 'right', image_12_16_r4 : 'right'}

    user_id, gender = test_instructions()
    results_dict = ans_test(images_list, images_dict, empty_image)
    
    combined_dict = {'user_id': user_id, 'gender': gender}
    combined_dict.update(results_dict) #adds the dictionary containing the results from the test to the dictionary containing the user ID and gender
    
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdNAOKUy2gKIWuvByvbJV8gLC7YTaaAxeKAq_7BEiw0TErzRw/viewform?usp=sf_link'
    send_to_google_form(combined_dict, form_url)
    
    print('The End. Thank you :>')
    #display(HTML('<p style="text-align: center; font-size: 22px">Ranking you against other people...</p>'))

    #ranking_df = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vTX3ITUBRrokx5fQ76pNA82N07Icw7NowPzFE6ifjyaffVV2cXA74ZC6y37-o1Dk7JipjlNOgla8jWN/pubhtml', usecols=['B', 'G'])
    #print(rankings_df)
    #rankings = ranking_df['total_correct'].rank(method='min', ascending=False)
    #ranking = rankings[ranking_df['total_correct'] == total_correct].iloc[0]
    #total_entries = len(ranking_df)
    #max_value = ranking_df['total_correct'].max()

    #time.sleep(2)
    #clear_output(wait=False)
    #display(HTML(f"<p style='text-align: center; font-size: 18px'>Your score in this test is {total_correct}.<br>You are ranked "
             #f"{ranking:.0f} out of {total_entries} testees.<br>Currently the highest score is {max_value}.<br>"
             #f"Feel free to try the test again for a better result!</p>"))

                    
if __name__ == '__main__':
    main()

DATA CONSENT INFORMATION:

Please read:

We wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload.
